In [1]:
#This performs Hierarchical modeling on a partially-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import skellam
import scipy.integrate as integrate
import scipy.special as special

In [2]:
#import saved data
pd_subsampled_x=pd.read_csv("subsampled.csv")
x= pd_subsampled_x.values
size=np.shape(x)

In [3]:
L=1000      #length of the chain
#I=size[0]    #[0,I-1] is the observation window
#M=size[1]    #number of individuals
#initializations
M=3
I=100
A=np.zeros((L,M))               #birth parameter
B=np.zeros((L,M))               #death parameter
alpha=np.zeros((L,M))           #alpha parameter
beta=np.zeros((L,M))            #beta parameter
a_A=np.zeros(L+1)               #birth shape hyperparameter
a_B=np.zeros(L+1)               #death shape hyperparameter
b_A=np.zeros(L+1)               #birth scale hyperparameter
b_B=np.zeros(L+1)               #death scale hyperparameter
a_alpha=np.zeros(L+1)           #alpha shape hyperparameter
a_beta=np.zeros(L+1)            #beta shape hyperparameter
b_alpha=np.zeros(L+1)           #alpha scale hyperparameter
b_beta=np.zeros(L+1)            #beta scale hyperparameter
r=np.zeros((2,I-1,M))     #reaction numbers
sum_x=np.zeros(M)   
var_a_A=2                      #birth shape hyperparameter proposal variance
var_a_B=2                       #death shape hyperparameter proposal variance
var_alpha=2                       #birth shape hyperparameter proposal variance
var_beta=2                       #death shape hyperparameter proposal variance
var_a_alpha=2
var_a_beta=2
tune=200                        #reaction proposal tuning parameter
#initial conditions
A[0,:]=1
B[0,:]=1
alpha[0,:]=1
beta[0,:]=1
a_A[0]=1
b_A[0]=1 
a_B[0]=1
b_B[0]=1
a_alpha[0]=1
b_alpha[0]=1
a_beta[0]=1
b_beta[0]=1

alpha[0,:]=np.load('alpha2.npy')
beta[0,:]=np.load('beta2.npy')

In [4]:
def init_reaction(x,s):         #initialize reaction numbers           
    q=np.zeros([2,s-1])
    for i in range(s-1):
        if x[i+1]-x[i]>0:
            q[0,i]=np.floor((4/3)*(x[i+1]-x[i]))
            q[1,i]=np.floor((1/3)*(x[i+1]-x[i]))
        else:
            q[0,i]=-np.floor((1/3)*(x[i+1]-x[i]))
            q[1,i]=-np.floor((4/3)*(x[i+1]-x[i]))
    r=q.astype(int)        
    return r  

def accept_rate_react(x,r,prop,i,A,B,jump,b,alpha,beta):        #acceptance rate for reaction numbers
    lambda_prop=1+(prop[0]**2)/b
    lambda_cur=1+(r[0]**2)/b
    delay_factor=int_gammainc(alpha,beta,i)
    prop_like=np.log(poisson.pmf(prop[0],A*delay_factor)+1e-300)+np.log(poisson.pmf(prop[1],0.5*B*(x[0]+x[1]))+1e-300)+np.log(skellam.pmf(abs(jump),lambda_prop,lambda_prop)+1e-300)
    current_like=np.log(poisson.pmf(r[0],A*delay_factor)+1e-300)+np.log(poisson.pmf(r[1],0.5*B*(x[0]+x[1]))+1e-300)+np.log(skellam.pmf(abs(jump),lambda_cur,lambda_cur)+1e-300)
    rate=np.minimum(1,np.exp(prop_like-current_like))
    return rate

def a_lik(A,a,b):           #shape hyperparameter likelihood function 
    sum1=0
    M=len(A)
    for s in range(M):
        sum1+=np.log(A[s])
    sum1*=(a-1)
    q=-M*np.log(math.gamma(a))+M*a*np.log(b)+sum1
    return q

def alpha_lik(alpha,beta,A,a,b,I,r):      #alpha loglikelihood
    sum_int_gammainc=0
    sum_log_gammainc=0
    for jj in range(I-1):
        kappa=int_gammainc(alpha,beta,jj)
        log_kappa=r[jj]*np.log(kappa+1e-300)
        sum_int_gammainc+=kappa
        sum_log_gammainc+=log_kappa
    q=sum_log_gammainc-(A*sum_int_gammainc)+((a-1)*np.log(alpha))-alpha*b    
    return q

def beta_lik(alpha,beta,A,a,b,I,r):    #beta loglikelihood
    sum_int_gammainc=0
    sum_log_gammainc=0
    for jj in range(I-1):
        kappa=int_gammainc(alpha,beta,jj)
        log_kappa=r[jj]*np.log(kappa+1e-300)
        sum_int_gammainc+=kappa
        sum_log_gammainc+=log_kappa
    q=sum_log_gammainc-(A*sum_int_gammainc)+((a-1)*np.log(beta))-beta*b    
    return q

def int_gammainc(alpha,beta,m):
    k,s=integrate.quad(lambda x: gamma.cdf(x,alpha,beta**(-1)),m,m+1)
    return k

In [5]:
#initialize number of reactions
for s in range(M):
    r[:,:,s]= init_reaction(x[:,s],I)
    sum_x[s]=x[0,s]+x[I-1,s]
    for j in range(I-2):
        sum_x[s]+=2*x[j+1,s]
    sum_x[s]=0.5*sum_x[s]    
#r=np.load('react2.npy') 
#print(r)

In [6]:
#perform sampling       
for i in range(L-1): 
    for s in range(M):
        #gamma incomplete function values
        sum_int_gammainc=0
        for j in range(I-1):
            sum_int_gammainc+=int_gammainc(alpha[i,s],beta[i,s],j)
        #print(sum_int_gammainc)
         
        #sample parameters
        A[i+1,s]=np.random.gamma(np.sum(r[0,:,s])+a_A[i],(sum_int_gammainc+b_A[i])**(-1))
        B[i+1,s]=np.random.gamma(np.sum(r[1,:,s])+a_B[i],(sum_x[s]+b_B[i])**(-1))
        #print('pass A,B')
              
        #update parameter alpha
        alpha_prop=-1
        while alpha_prop<0 or alpha_prop>170:
            alpha_prop=alpha[i,s]+np.random.normal(0,var_alpha)
        a=alpha_lik(alpha_prop,beta[i,s],A[i+1,s],a_alpha[i],b_alpha[i],I,r[0,:,s]) 
        b=alpha_lik(alpha[i,s],beta[i,s],A[i+1,s],a_alpha[i],b_alpha[i],I,r[0,:,s])
        rate=np.minimum(1,np.exp(a-b))
        #print(rate)
        if np.random.uniform(0,1,1)<rate:    
            alpha[i+1,s]=alpha_prop
        else:
            alpha[i+1,s]=alpha[i,s]
        #print('pass alpha')
        
        #update parameter beta
        beta_prop=-1
        while beta_prop<0 or beta_prop>170:
            beta_prop=beta[i,s]+np.random.normal(0,var_beta)
        a=beta_lik(alpha[i+1,s],beta_prop,A[i+1,s],a_beta[i],b_beta[i],I,r[0,:,s])   
        b=beta_lik(alpha[i+1,s],beta[i,s],A[i+1,s],a_beta[i],b_beta[i],I,r[0,:,s])
        rate=np.minimum(1,np.exp(a-b))
        #print(rate)
        if np.random.uniform(0,1,1)<rate:    
            beta[i+1,s]=beta_prop
        else:
            beta[i+1,s]=beta[i,s]
        #print('pass beta')
        
        
        #update reaction numbers
        for j in range(I-1):
            r_vprop=np.array([-1,-1])
            lamb=1+(r[0,j,s]**2)/tune                 #tuning parameter suggested in Boy's paper 
            while r_vprop[0]<0 or r_vprop[1]<0:
                jump=np.random.poisson(lamb)-np.random.poisson(lamb)
                r_prop=r[0,j,s]+jump
                r_vprop=[r_prop, r_prop-(x[j+1,s]-x[j,s])]    
            rate=accept_rate_react([x[j,s],x[j+1,s]],r[:,j,s],r_vprop,j,A[i+1,s],B[i+1,s],jump,tune,alpha[i+1,s],beta[i+1,s]) 
            if np.random.uniform(0,1,1)<rate:
                r[:,j,s]=r_vprop    
        #print('pass r')   
        
    #sample hyperparameters a_A and a_B
    a_Aprop=-1
    while a_Aprop<0 or a_Aprop>170:
        a_Aprop=a_A[i]+np.random.normal(0,var_a_A)
    a=a_lik(A[i+1,:],a_Aprop,b_A[i]) 
    b=a_lik(A[i+1,:],a_A[i],b_A[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_A[i+1]=a_Aprop
    else:
        a_A[i+1]=a_A[i]
        
    a_Bprop=-1
    while a_Bprop<0 or a_Bprop>170:
        a_Bprop=a_B[i]+np.random.normal(0,var_a_B)
    a=a_lik(B[i+1,:],a_Bprop,b_B[i]) 
    b=a_lik(B[i+1,:],a_B[i],b_B[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_B[i+1]=a_Bprop
    else:
        a_B[i+1]=a_B[i]    
    #print('pass a_A,B')
    
    #sample hyperparameters a_alpha and a_beta
    a_alphaprop=-1
    while a_alphaprop<0 or a_alphaprop>170:
        a_alphaprop=a_alpha[i]+np.random.normal(0,var_a_alpha)
    a=a_lik(alpha[i+1,:],a_alphaprop,b_alpha[i]) 
    b=a_lik(alpha[i+1,:],a_alpha[i],b_alpha[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_alpha[i+1]=a_alphaprop
    else:
        a_alpha[i+1]=a_alpha[i]
        
    a_betaprop=-1
    while a_betaprop<0 or a_betaprop>170:
        a_betaprop=a_beta[i]+np.random.normal(0,var_a_beta)
    a=a_lik(beta[i+1,:],a_betaprop,b_beta[i]) 
    b=a_lik(beta[i+1,:],a_beta[i],b_beta[i])
    rate=np.minimum(1,np.exp(a-b))
    if np.random.uniform(0,1,1)<rate:    
        a_beta[i+1]=a_betaprop
    else:
        a_beta[i+1]=a_beta[i]
    #print('pass a_alpha,beta')
    
    #sample hyperparameters b_A, b_B, b_alpha, b_beta
    sum1=np.sum(A[i+1,:])
    sum2=np.sum(B[i+1,:])
    sum3=np.sum(alpha[i+1,:])
    sum4=np.sum(beta[i+1,:])
    b_A[i+1]=np.random.gamma(M*a_A[i+1],sum1**(-1))
    b_B[i+1]=np.random.gamma(M*a_B[i+1],sum2**(-1))
    b_alpha[i+1]=np.random.gamma(M*a_alpha[i+1],sum3**(-1))
    b_beta[i+1]=np.random.gamma(M*a_beta[i+1],sum4**(-1))
    #print('pass b_A,B,alpha,beta')
        
    if i%10 == 0:  
        print(i+1,'birth',a_A[i+1],b_A[i+1],'death',a_B[i+1],b_B[i+1]) 
        print(i+1,'alpha',a_alpha[i+1],b_alpha[i+1],'beta',a_beta[i+1],b_beta[i+1]) 
        print("%%%%%%%%%%%%%%%%%next%%%%%%%%%%%%%%%%")
    elif i == L-2:
        print(i+1,'birth',a_A[i+1],b_A[i+1],'death',a_B[i+1],b_B[i+1]) 
        print(i+1,'alpha',a_alpha[i+1],b_alpha[i+1],'beta',a_beta[i+1],b_beta[i+1]) 
        print("%%%%%%%%%%%%%%%%%done%%%%%%%%%%%%%%%%")

1 birth 1.0 0.1534099638316179 death 0.8356801274909419 1.8862490400802594
1 alpha 2.060780076537152 1.413653864019255 beta 1.0 0.4505551065540977
%%%%%%%%%%%%%%%%%next%%%%%%%%%%%%%%%%
11 birth 1.1355126398204756 0.41186970938934114 death 1.0240822921405788 5.0972486877946075
11 alpha 5.523320020031434 2.105805160712806 beta 1.1307841945323354 0.4390948523825874
%%%%%%%%%%%%%%%%%next%%%%%%%%%%%%%%%%
21 birth 2.8206754133378142 1.195440275054632 death 4.158471116559137 32.947774753667936
21 alpha 7.803555162821947 2.974992228239485 beta 0.9397771204849321 0.12666160683667363
%%%%%%%%%%%%%%%%%next%%%%%%%%%%%%%%%%
31 birth 5.379439820449292 1.6825438591974364 death 1.230763321341695 8.182957045282338
31 alpha 10.818509419111875 4.344954028655923 beta 0.7851817446027469 0.426201473156848
%%%%%%%%%%%%%%%%%next%%%%%%%%%%%%%%%%
41 birth 9.17002350518447 2.2455799905040705 death 5.923445691255309 39.524076814888424
41 alpha 14.0908262226219 4.009726893771449 beta 3.1553620745050903 0.455672365

KeyboardInterrupt: 

In [ ]:
#save the simulated values into .csv files
np.save('alpha2.npy',alpha[i-1,:])
np.save('beta2.npy',beta[i-1,:])
np.save('react2.npy',r) 
columns=' '
np.savetxt("A21.csv", A, delimiter=",",header=columns)
np.savetxt("B21.csv", B, delimiter=",",header=columns)
np.savetxt("alpha21.csv", alpha, delimiter=",",header=columns)
np.savetxt("beta21.csv", beta, delimiter=",",header=columns)
np.savetxt("a_A21.csv", a_A, delimiter=",",header=columns)
np.savetxt("b_A21.csv", b_A, delimiter=",",header=columns)
np.savetxt("a_B21.csv", a_B, delimiter=",",header=columns)
np.savetxt("b_B21.csv", b_B, delimiter=",",header=columns)
np.savetxt("a_alpha21.csv", a_alpha, delimiter=",",header=columns)
np.savetxt("b_alpha21.csv", b_alpha, delimiter=",",header=columns)
np.savetxt("a_beta21.csv", a_beta, delimiter=",",header=columns)
np.savetxt("b_beta21.csv", b_beta, delimiter=",",header=columns)

In [ ]:
np.save('r.npy',r)

In [ ]:
np.save('r.npy',r)
np.save('A.npy',A)
np.save('B.npy',B)
np.save('alpha.npy',alpha)
np.save('beta.npy',beta)
np.save('a_A.npy',a_A)
np.save('a_B.npy',a_B)
np.save('b_A.npy',b_A)
np.save('b_B.npy',b_B)
np.save('a_alpha.npy',a_alpha)
np.save('a_beta.npy',a_beta)
np.save('b_alpha.npy',b_alpha)
np.save('b_beta.npy',b_beta)

In [ ]:
r=np.load('r.npy')
A=np.load('A.npy')
B=np.load('B.npy')
alpha=np.load('alpha.npy')
beta=np.load('beta.npy')
a_A=np.load('a_A.npy')
a_B=np.load('a_B.npy')
b_A=np.load('b_A.npy')
b_B=np.load('b_B.npy')
a_alpha=np.load('a_alpha.npy')
a_beta=np.load('a_beta.npy')
b_alpha=np.load('b_alpha.npy')
b_beta=np.load('b_beta.npy')

In [ ]:
A=A[:i,:]
B=B[:i,:]
alpha=alpha[:i,:]
beta=beta[:i,:]
a_A=a_A[:i]
a_B=a_B[:i]
b_A=b_A[:i]
b_B=b_B[:i]
a_alpha=a_alpha[:i]
a_beta=a_beta[:i]
b_alpha=b_alpha[:i]
b_beta=b_beta[:i]

In [ ]:
plt.hist(A[:,1])
plt.show()
plt.hist(A[:,2])
plt.show()
plt.hist(A[:,3])
plt.show()
plt.hist(A[:,4])
plt.show()
plt.hist(A[:,5])
plt.show()

In [ ]:
plt.hist(B[:,1])
plt.show()
plt.hist(B[:,2])
plt.show()
plt.hist(B[:,3])
plt.show()
plt.hist(B[:,4])
plt.show()
plt.hist(B[:,5])
plt.show()

In [ ]:
plt.hist(alpha[:,1])
plt.show()
plt.hist(alpha[:,2])
plt.show()
plt.hist(alpha[:,3])
plt.show()
plt.hist(alpha[:,4])
plt.show()
plt.hist(alpha[:,5])
plt.show()

In [ ]:
plt.hist(beta[:,1])
plt.show()
plt.hist(beta[:,2])
plt.show()
plt.hist(beta[:,3])
plt.show()
plt.hist(beta[:,4])
plt.show()
plt.hist(beta[:,5])
plt.show()

In [ ]:
plt.hist(a_alpha)
plt.show()
plt.hist(b_alpha)
plt.show()

In [ ]:
plt.hist(a_beta)
plt.show()
plt.hist(b_beta)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(a_A,lags=len(a_A)-1)
plot_acf(b_A,lags=len(b_A)-1)
plt.show

In [ ]:
r[:,:,6]

In [ ]:
tA=np.load('trueA.npy')

In [ ]:
A[i+1,:]

In [ ]:
B[i+1,:]

In [ ]:
a_A[i+1]

In [ ]:
b_A[i+1]

In [ ]:
import numpy as np
np.log(1e-300)

In [ ]:
B[:,4]